## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [2]:
from allennlp.data.token_indexers import TokenIndexer, PretrainedTransformerIndexer
from allennlp.data.tokenizers import Token, Tokenizer, PretrainedTransformerTokenizer

import nltk
#nltk.download('punkt')
import numpy as np
from os import listdir
from os.path import join as pathjoin
import torch
import torch.nn as nn
from torch.nn import functional as F
import tqdm

from minGPT.mingpt.model import GPT, GPTConfig
from minGPT.mingpt.trainer import Trainer, TrainerConfig
# make deterministic
from minGPT.mingpt.utils import sample, set_seed
set_seed(42)

In [3]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
transformer_model = 'DeepPavlov/rubert-base-cased'

In [4]:
import math
from torch.utils.data import Dataset


def detokenize(tokens):
    return ' '.join([str(x) for x in tokens[1:-1]]).replace(' ##', '')

class BPEDataset(Dataset):
    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


In [5]:
block_size = 128
tokenizer = PretrainedTransformerTokenizer(transformer_model)
#indexer = PretrainedTransformerIndexer(transformer_model)

In [10]:
def train_gpt_generator(train_text_file, state_dict_file, n_layer=8, n_head=8, n_embd=512,
                        max_epochs=2, batch_size=256):
    text_sentences = nltk.tokenize.sent_tokenize(open(train_text_file, 'r').read())
    tokens = np.concatenate([tokenizer.tokenize(sent)[1:-1] for sent in text_sentences])
    tokens = [str(token) for token in tokens]
    train_dataset = BPEDataset(tokens, block_size) 
    print('dataset is read')
    
    mconf = GPTConfig(
        train_dataset.vocab_size, train_dataset.block_size,
        n_layer=n_layer, n_head=n_head, n_embd=n_embd
    )
    model = GPT(mconf)
    print('model is built')
    tconf = TrainerConfig(
        max_epochs=max_epochs, batch_size=batch_size, learning_rate=6e-4,
        lr_decay=True, warmup_tokens=batch_size*20, final_tokens=2*len(train_dataset)*block_size,
        num_workers=4
    )
    trainer = Trainer(model, train_dataset, None, tconf)
    trainer.train()
    torch.save(model.state_dict(), state_dict_file)

In [11]:
GENRE_DATA_DIR = '/home/mlepekhin/data/genre'
GPT_MODELS_DIR = '/home/mlepekhin/models/mini_gpt_bpe'
LANG = 'ru_lemma'
!ls /home/mlepekhin/models/mini_gpt_bpe

en  ru	ru_lemma


In [12]:
#train_gpt_generator(
#        pathjoin(GENRE_DATA_DIR, LANG, 'A1.txt'),
#        pathjoin(GPT_MODELS_DIR, LANG, 'A1')
#)

In [ ]:
for train_text_file in tqdm.tqdm(listdir(pathjoin(GENRE_DATA_DIR, LANG))):
    label = train_text_file[:-4]
    train_gpt_generator(
        pathjoin(GENRE_DATA_DIR, LANG, train_text_file),
        pathjoin(GPT_MODELS_DIR, LANG, label)
    )

  0%|          | 0/10 [00:00<?, ?it/s]

data has 187706 characters, 11591 unique.
dataset is read
model is built



  0%|          | 0/733 [00:00<?, ?it/s]/home/mlepekhin/anaconda3/envs/mlepekhin_research/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

epoch 1 iter 0: train loss 9.48513. lr 5.999995e-04:   0%|          | 0/733 [00:07<?, ?it/s]
epoch 1 iter 0: train loss 9.48513. lr 5.999995e-04:   0%|          | 1/733 [00:07<1:28:55,  7.29s/it]
epoch 1 iter 1: train loss 8.67772. lr 5.999977e-04:   0%|          | 1/733 [00:07<1:28:55,  7.29s/it]
epoch 1 iter 1: train loss 8.67772. lr 5.999977e-04:   0%|          | 2/733 [00:07<1:03:26,  5.21s/it]
epoch 1 iter 2: train loss 8.19510. lr 5.999944e-04:   0%|          | 2/733 [00:07<1:03:26,  5.21s/it]
epoch 1 iter 2: train loss 8.19510. lr 5.999944e-04:   0%|          | 3/733 [00:07<45:32,  3.74s/it]  
epoch 1 iter 3: train loss 7.90512. lr 

epoch 1 iter 37: train loss 6.08536. lr 5.990131e-04:   5%|▌         | 37/733 [00:19<04:14,  2.73it/s]
epoch 1 iter 37: train loss 6.08536. lr 5.990131e-04:   5%|▌         | 38/733 [00:19<04:05,  2.83it/s]
epoch 1 iter 38: train loss 6.03918. lr 5.989602e-04:   5%|▌         | 38/733 [00:19<04:05,  2.83it/s]
epoch 1 iter 38: train loss 6.03918. lr 5.989602e-04:   5%|▌         | 39/733 [00:19<03:58,  2.91it/s]
epoch 1 iter 39: train loss 6.07850. lr 5.989061e-04:   5%|▌         | 39/733 [00:20<03:58,  2.91it/s]
epoch 1 iter 39: train loss 6.07850. lr 5.989061e-04:   5%|▌         | 40/733 [00:20<03:54,  2.96it/s]
epoch 1 iter 40: train loss 6.12726. lr 5.988505e-04:   5%|▌         | 40/733 [00:20<03:54,  2.96it/s]
epoch 1 iter 40: train loss 6.12726. lr 5.988505e-04:   6%|▌         | 41/733 [00:20<03:50,  3.00it/s]
epoch 1 iter 41: train loss 6.03930. lr 5.987935e-04:   6%|▌         | 41/733 [00:20<03:50,  3.00it/s]
epoch 1 iter 41: train loss 6.03930. lr 5.987935e-04:   6%|▌         | 42

epoch 1 iter 75: train loss 5.65926. lr 5.960425e-04:  10%|█         | 76/733 [00:31<03:29,  3.14it/s]
epoch 1 iter 76: train loss 5.60277. lr 5.959377e-04:  10%|█         | 76/733 [00:32<03:29,  3.14it/s]
epoch 1 iter 76: train loss 5.60277. lr 5.959377e-04:  11%|█         | 77/733 [00:32<03:51,  2.83it/s]
epoch 1 iter 77: train loss 5.60534. lr 5.958315e-04:  11%|█         | 77/733 [00:32<03:51,  2.83it/s]
epoch 1 iter 77: train loss 5.60534. lr 5.958315e-04:  11%|█         | 78/733 [00:32<03:44,  2.91it/s]
epoch 1 iter 78: train loss 5.56525. lr 5.957240e-04:  11%|█         | 78/733 [00:32<03:44,  2.91it/s]
epoch 1 iter 78: train loss 5.56525. lr 5.957240e-04:  11%|█         | 79/733 [00:32<03:39,  2.98it/s]
epoch 1 iter 79: train loss 5.58142. lr 5.956151e-04:  11%|█         | 79/733 [00:33<03:39,  2.98it/s]
epoch 1 iter 79: train loss 5.58142. lr 5.956151e-04:  11%|█         | 80/733 [00:33<03:35,  3.03it/s]
epoch 1 iter 80: train loss 5.56352. lr 5.955048e-04:  11%|█         | 80

...

epoch 2 iter 590: train loss 0.19487. lr 1.840983e-04:  25%|██▌       | 590/2341 [04:29<13:44,  2.12it/s]
epoch 2 iter 590: train loss 0.19487. lr 1.840983e-04:  25%|██▌       | 591/2341 [04:29<13:37,  2.14it/s]
epoch 2 iter 591: train loss 0.19056. lr 1.839126e-04:  25%|██▌       | 591/2341 [04:29<13:37,  2.14it/s]
epoch 2 iter 591: train loss 0.19056. lr 1.839126e-04:  25%|██▌       | 592/2341 [04:29<13:23,  2.18it/s]
epoch 2 iter 592: train loss 0.19097. lr 1.837269e-04:  25%|██▌       | 592/2341 [04:29<13:23,  2.18it/s]
epoch 2 iter 592: train loss 0.19097. lr 1.837269e-04:  25%|██▌       | 593/2341 [04:29<13:13,  2.20it/s]
epoch 2 iter 593: train loss 0.18563. lr 1.835413e-04:  25%|██▌       | 593/2341 [04:30<13:13,  2.20it/s]
epoch 2 iter 593: train loss 0.18563. lr 1.835413e-04:  25%|██▌       | 594/2341 [04:30<13:10,  2.21it/s]
epoch 2 iter 594: train loss 0.18518. lr 1.833558e-04:  25%|██▌       | 594/2341 [04:30<13:10,  2.21it/s]
epoch 2 iter 594: train loss 0.18518. lr 1.833

epoch 2 iter 627: train loss 0.19193. lr 1.772623e-04:  27%|██▋       | 628/2341 [04:46<12:53,  2.22it/s]
epoch 2 iter 628: train loss 0.18057. lr 1.770786e-04:  27%|██▋       | 628/2341 [04:46<12:53,  2.22it/s]
epoch 2 iter 628: train loss 0.18057. lr 1.770786e-04:  27%|██▋       | 629/2341 [04:46<12:44,  2.24it/s]
epoch 2 iter 629: train loss 0.18812. lr 1.768949e-04:  27%|██▋       | 629/2341 [04:46<12:44,  2.24it/s]
epoch 2 iter 629: train loss 0.18812. lr 1.768949e-04:  27%|██▋       | 630/2341 [04:46<12:44,  2.24it/s]
epoch 2 iter 630: train loss 0.18895. lr 1.767113e-04:  27%|██▋       | 630/2341 [04:47<12:44,  2.24it/s]
epoch 2 iter 630: train loss 0.18895. lr 1.767113e-04:  27%|██▋       | 631/2341 [04:47<12:45,  2.23it/s]
epoch 2 iter 631: train loss 0.18143. lr 1.765278e-04:  27%|██▋       | 631/2341 [04:47<12:45,  2.23it/s]
epoch 2 iter 631: train loss 0.18143. lr 1.765278e-04:  27%|██▋       | 632/2341 [04:47<12:44,  2.23it/s]
epoch 2 iter 632: train loss 0.18262. lr 1.763

epoch 2 iter 665: train loss 0.17903. lr 1.703205e-04:  28%|██▊       | 665/2341 [05:03<13:08,  2.13it/s]
epoch 2 iter 665: train loss 0.17903. lr 1.703205e-04:  28%|██▊       | 666/2341 [05:03<13:03,  2.14it/s]
epoch 2 iter 666: train loss 0.18436. lr 1.701389e-04:  28%|██▊       | 666/2341 [05:03<13:03,  2.14it/s]
epoch 2 iter 666: train loss 0.18436. lr 1.701389e-04:  28%|██▊       | 667/2341 [05:03<12:56,  2.15it/s]
epoch 2 iter 667: train loss 0.18984. lr 1.699574e-04:  28%|██▊       | 667/2341 [05:04<12:56,  2.15it/s]
epoch 2 iter 667: train loss 0.18984. lr 1.699574e-04:  29%|██▊       | 668/2341 [05:04<12:52,  2.17it/s]
epoch 2 iter 668: train loss 0.18096. lr 1.697760e-04:  29%|██▊       | 668/2341 [05:04<12:52,  2.17it/s]
epoch 2 iter 668: train loss 0.18096. lr 1.697760e-04:  29%|██▊       | 669/2341 [05:04<12:47,  2.18it/s]
epoch 2 iter 669: train loss 0.18504. lr 1.695946e-04:  29%|██▊       | 669/2341 [05:05<12:47,  2.18it/s]
epoch 2 iter 669: train loss 0.18504. lr 1.695

epoch 2 iter 702: train loss 0.18406. lr 1.636423e-04:  30%|███       | 703/2341 [05:20<12:13,  2.23it/s]
epoch 2 iter 703: train loss 0.18227. lr 1.634630e-04:  30%|███       | 703/2341 [05:20<12:13,  2.23it/s]
epoch 2 iter 703: train loss 0.18227. lr 1.634630e-04:  30%|███       | 704/2341 [05:20<12:12,  2.24it/s]
epoch 2 iter 704: train loss 0.18718. lr 1.632837e-04:  30%|███       | 704/2341 [05:21<12:12,  2.24it/s]
epoch 2 iter 704: train loss 0.18718. lr 1.632837e-04:  30%|███       | 705/2341 [05:21<12:12,  2.23it/s]
epoch 2 iter 705: train loss 0.17259. lr 1.631045e-04:  30%|███       | 705/2341 [05:21<12:12,  2.23it/s]
epoch 2 iter 705: train loss 0.17259. lr 1.631045e-04:  30%|███       | 706/2341 [05:21<12:09,  2.24it/s]
epoch 2 iter 706: train loss 0.19014. lr 1.629253e-04:  30%|███       | 706/2341 [05:22<12:09,  2.24it/s]
epoch 2 iter 706: train loss 0.19014. lr 1.629253e-04:  30%|███       | 707/2341 [05:22<12:31,  2.17it/s]
epoch 2 iter 707: train loss 0.18363. lr 1.627

epoch 2 iter 740: train loss 0.17969. lr 1.568712e-04:  32%|███▏      | 740/2341 [05:37<12:36,  2.12it/s]
epoch 2 iter 740: train loss 0.17969. lr 1.568712e-04:  32%|███▏      | 741/2341 [05:37<12:41,  2.10it/s]
epoch 2 iter 741: train loss 0.18416. lr 1.566943e-04:  32%|███▏      | 741/2341 [05:38<12:41,  2.10it/s]
epoch 2 iter 741: train loss 0.18416. lr 1.566943e-04:  32%|███▏      | 742/2341 [05:38<12:41,  2.10it/s]
epoch 2 iter 742: train loss 0.17503. lr 1.565174e-04:  32%|███▏      | 742/2341 [05:38<12:41,  2.10it/s]
epoch 2 iter 742: train loss 0.17503. lr 1.565174e-04:  32%|███▏      | 743/2341 [05:38<12:36,  2.11it/s]
epoch 2 iter 743: train loss 0.17809. lr 1.563406e-04:  32%|███▏      | 743/2341 [05:39<12:36,  2.11it/s]
epoch 2 iter 743: train loss 0.17809. lr 1.563406e-04:  32%|███▏      | 744/2341 [05:39<12:34,  2.12it/s]
epoch 2 iter 744: train loss 0.16845. lr 1.561638e-04:  32%|███▏      | 744/2341 [05:39<12:34,  2.12it/s]
epoch 2 iter 744: train loss 0.16845. lr 1.561

In [ ]:
print(1)